In [70]:
import random

from tqdm import tqdm
from datasets import load_from_disk, DatasetDict

In [71]:
wiki_datasets = load_from_disk("/home/ubuntu/workspace/data/wiki_preprocessed_droped")
train_dataset = load_from_disk("/home/ubuntu/workspace/data/new_train_preprocessed")
wiki_datasets.load_elasticsearch_index("text", host="localhost", port="9200", es_index_name="wikipedia_contexts")

In [72]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title', 'chunks'],
        num_rows: 3351
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title', 'chunks'],
        num_rows: 841
    })
})

In [73]:
dicts = []
for wiki in tqdm(wiki_datasets):
  wiki_dick = {}
  wiki_dick['text'] = wiki['text']
  wiki_dick['meta'] = {
    'title': wiki['title'],
    'document_id': wiki['document_id']
  }
  dicts.append(wiki_dick)

100%|██████████| 55962/55962 [00:08<00:00, 6380.59it/s]


In [74]:
dict_id = []
for wiki in tqdm(dicts):
  dict_id.append(wiki['meta']['document_id'])

100%|██████████| 55962/55962 [00:00<00:00, 1753197.89it/s]


In [75]:
error_context = []

def drop_not_doc(datasets):
  def get_distance(a, b):
    return abs(a - b)
  def change_doc_id(example):
    # if example['document_id'] not in not_include:
    #   return example
    doc = wiki_datasets.get_nearest_examples("text", example['context'][:1024], k=1)
    document_id = doc[1]['document_id'][0]
    title = doc[1]['title'][0]
    context_list = doc[1]['text'][0].split(example['answers']['text'][0])
    if len(context_list) == 1:
      error_context.append(example, context_list)
      return example
    distance = 100000
    location = -len(example['answers']['text'][0])
    for context in context_list:
      old_location = location
      location += len(context) + len(example['answers']['text'][0])
      new_distance = get_distance(location, example['answers']['answer_start'][0])
      if distance > new_distance:
        distance = new_distance
      else:
        location = old_location
        break
    example['answers']['answer_start'][0] = location
    example['document_id'] = document_id
    example['title'] = title
    example['context'] = example['answers']['text'][0].join(context_list)
    return example
  return datasets.map(change_doc_id)

In [76]:
new_train_dataset = drop_not_doc(train_dataset['train'])
new_valid_dataset = drop_not_doc(train_dataset['validation'])

  0%|          | 0/3351 [00:00<?, ?ex/s]

  0%|          | 0/841 [00:00<?, ?ex/s]

In [77]:
len(error_context)

0

In [78]:
search_error = new_train_dataset.filter(lambda example: example['context'][example['answers']['answer_start'][0]:example['answers']['answer_start'][0]+len(example['answers']['text'][0])] != example['answers']['text'][0])

  0%|          | 0/4 [00:00<?, ?ba/s]

In [79]:
search_error

Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title', 'chunks'],
    num_rows: 0
})

In [80]:
data = {
  'train': new_train_dataset,
  'validation': new_valid_dataset
}
train = DatasetDict(data)
train.save_to_disk('new_gen_dataset') # 저장위치